# Redes neuronales: TabNet

## 1.- Descripción

TabNet es un modelo basado en redes neuronales desarrollado específicamente para trabajar con datos tabulares. Combina la capacidad de aprendizaje profundo con la flexibilidad de los árboles de decisión, lo que le permite manejar eficientemente tanto relaciones lineales como no lineales. El modelo se distingue por su arquitectura única que utiliza atención dirigida (decision-aware attention) para seleccionar las características más relevantes durante el entrenamiento, mejorando así la interpretabilidad y el rendimiento.

En este proyecto, TabNet se utiliza para predecir si un préstamo será aceptado o rechazado por un banco, basándose en un conjunto de características del solicitante, como su historial financiero, información personal y otros factores relevantes. Este tipo de problema se ajusta bien al enfoque de TabNet, ya que maneja datos tabulares y es capaz de identificar patrones complejos en los datos mientras mantiene la capacidad de interpretabilidad.

## 2.- Implementación


In [6]:
# Importar las librerías necesarias
#Están incluidas librerías que fueron usadas como prueba
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import torch.optim as optim

In [7]:
# Cargar los datos
df = pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\new\train_balanced_processed.csv")
df_test= pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\new\test_nolabel_processed.csv")


In [8]:
df_test_features = df_test.drop(columns=['id'], errors='ignore') 


In [9]:
# Balancear el dataset con undersampling
df_majority = df[df["Accept"] == 1]
df_minority = df[df["Accept"] == 0]
df_majority_downsampled = resample(df_majority, 
                                   replace=False,
                                   n_samples=len(df_minority),
                                   random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [10]:
# Separar X e y
X = df_balanced.drop('Accept', axis=1)
y = df_balanced['Accept']

In [11]:
train_columns = X.columns

In [12]:
# Codificar columnas categóricas
le = LabelEncoder()
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    X[col] = le.fit_transform(X[col].astype(str))

In [13]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns

categorical_columns_test = [col for col in categorical_columns_test if col != 'id']
for col in categorical_columns_test:
    if col in categorical_columns:  # Solo transformar las columnas que están en df_train
        # Reemplaza las categorías desconocidas con 'Unknown' si es necesario
        if 'Unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Unknown')
        
        # Reemplaza las categorías desconocidas con 'Unknown' y aplica la transformación
        df_test[col] = df_test[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
        df_test[col] = le.transform(df_test[col])

In [14]:
categorical_columns_test = df_test_features.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns_test:
    df_test_features[col] = le.fit_transform(df_test_features[col].astype(str))


In [15]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
df_test_imputed = imputer.fit_transform(df_test_features)

In [16]:
# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
df_test_scaled = scaler.fit_transform(df_test_imputed)

In [17]:
# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## 3.- Evaluación y optimización


In [19]:
# Definir el modelo TabNet
tabnet = TabNetClassifier(optimizer_fn=optim.Adam, optimizer_params={'lr': 1e-4})


C:\Users\Day\anaconda3\envs\myenv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [20]:
# Entrenar el modelo
#A lo largo del entrenamiento se fueron modificando las diferentes características del modelo para buscar optimizarlo.
tabnet.fit(X_train=X_train, y_train=y_train, max_epochs=200, patience=30, batch_size=256, virtual_batch_size=128)

C:\Users\Day\anaconda3\envs\myenv\lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.95643 |  0:00:02s
epoch 1  | loss: 0.94579 |  0:00:04s
epoch 2  | loss: 0.91883 |  0:00:08s
epoch 3  | loss: 0.92037 |  0:00:10s
epoch 4  | loss: 0.90987 |  0:00:11s
epoch 5  | loss: 0.89254 |  0:00:13s
epoch 6  | loss: 0.89292 |  0:00:14s
epoch 7  | loss: 0.87194 |  0:00:16s
epoch 8  | loss: 0.85996 |  0:00:17s
epoch 9  | loss: 0.85092 |  0:00:19s
epoch 10 | loss: 0.84995 |  0:00:21s
epoch 11 | loss: 0.82351 |  0:00:22s
epoch 12 | loss: 0.82471 |  0:00:24s
epoch 13 | loss: 0.81286 |  0:00:25s
epoch 14 | loss: 0.82127 |  0:00:27s
epoch 15 | loss: 0.81824 |  0:00:29s
epoch 16 | loss: 0.81253 |  0:00:30s
epoch 17 | loss: 0.79482 |  0:00:32s
epoch 18 | loss: 0.79602 |  0:00:33s
epoch 19 | loss: 0.79518 |  0:00:35s
epoch 20 | loss: 0.77838 |  0:00:37s
epoch 21 | loss: 0.77687 |  0:00:38s
epoch 22 | loss: 0.7682  |  0:00:40s
epoch 23 | loss: 0.77013 |  0:00:41s
epoch 24 | loss: 0.76759 |  0:00:43s
epoch 25 | loss: 0.75074 |  0:00:45s
epoch 26 | loss: 0.77349 |  0:00:46s
e

In [21]:
# Predecir
y_pred = tabnet.predict(X_test)

In [22]:
y_pred_prob = tabnet.predict_proba(X_test)
y_pred = (y_pred_prob[:, 1] > 0.4).astype(int)  # Umbral de 0.4


In [23]:
# Hacer predicciones
y_test_pred = tabnet.predict(df_test_scaled)

In [24]:
# Evaluar
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[245 344]
 [130 452]]
              precision    recall  f1-score   support

           0       0.65      0.42      0.51       589
           1       0.57      0.78      0.66       582

    accuracy                           0.60      1171
   macro avg       0.61      0.60      0.58      1171
weighted avg       0.61      0.60      0.58      1171



In [25]:
df_test['ACCEPT'] = y_test_pred
print(df_test[['id', 'ACCEPT']].head())
missing_accept = df_test['ACCEPT'].isnull().sum()
print(missing_accept)


            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       1
2  b2c5181ac5b       0
3  e6cb54a9e6a       0
4  8eddf83466e       1
0


In [26]:
# Crear DataFrame con las predicciones (id y Accept)
df_result = df_test[['id']].copy()  # Agregar 'id' del conjunto de test
df_result['ACCEPT'] = y_test_pred
# Verifica las primeras filas del DataFrame
print(df_result.head())

            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       1
2  b2c5181ac5b       0
3  e6cb54a9e6a       0
4  8eddf83466e       1


In [27]:
# Guardar el resultado en un archivo CSV
df_result.to_csv("SUBMMIT_TABNET.csv", index=False)

print("¡Predicciones completas y archivo guardado como 'resultado_test.csv'!")

¡Predicciones completas y archivo guardado como 'resultado_test.csv'!


## 4.- Conclusión

El modelo TabNet, diseñado para trabajar con datos tabulares, logró una exactitud global del 60% en la predicción de la aceptación de préstamos bancarios. Si bien su rendimiento general es moderado, se observa un desempeño desigual entre las dos clases.

En particular, el modelo muestra una buena capacidad para identificar correctamente los préstamos aprobados (clase 1), con un recall del 78%, lo que significa que detecta de manera efectiva la mayoría de los casos positivos. Sin embargo, esta ventaja viene acompañada de un bajo rendimiento en la clasificación de los préstamos rechazados (clase 0), donde el recall fue solo del 42%, reflejando una alta tasa de falsos negativos en esta categoría.

Los valores de f1-score refuerzan este desequilibrio: 0.66 para la clase aprobada y 0.51 para la rechazada. Esto sugiere que, aunque el modelo favorece la identificación de solicitudes aprobadas, su precisión en detectar rechazos es limitada, lo que puede generar riesgos en un contexto bancario.

En conclusión, aunque TabNet ofrece ventajas en la detección de préstamos aprobados, su aplicación en escenarios reales requeriría ajustes para mejorar la sensibilidad hacia los casos rechazados. Sería recomendable considerar técnicas de balanceo de clases, ajuste de hiperparámetros o combinarlo con otros modelos para mejorar el equilibrio en las predicciones.